In [1]:
! pip install pyswarm

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4485 sha256=b4d03308b601224b85f738962f7371d28548fee7a89f3411744b6802517cd35f
  Stored in directory: c:\users\amlmk\appdata\local\pip\cache\wheels\bb\4f\ec\8970b83323e16aa95034da175454843947376614d6d5e9627f
Successfully built pyswarm


In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from pyswarm import pso

# # Load Iris dataset (you can replace this with your dataset)
# iris = load_iris()
# X = iris.data
# y = iris.target
# import pandas as pd
# from sklearn.datasets import load_iris

# # Load Iris dataset
# iris = load_iris()
# X = iris.data
# y = iris.target

# # Create a DataFrame
# df = pd.DataFrame(X, columns=iris.feature_names)

# # Get column names
# column_names = df.columns
# print(column_names)


In [9]:
import pandas as pd
diabetes_df = pd.read_csv(r"C:\Users\amlmk\Downloads\Diabetes Dataset\Diabetes Dataset\Dataset of Diabetes .csv")
diabetes_df

,ID,No_Pation,Gender,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI,CLASS
0,502,17975,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
1,735,34221,M,26,4.5,62,4.9,3.7,1.4,1.1,2.1,0.6,23.0,N
2,420,47975,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
3,680,87656,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
4,504,34223,M,33,7.1,46,4.9,4.9,1.0,0.8,2.0,0.4,21.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,200,454317,M,71,11.0,97,7.0,7.5,1.7,1.2,1.8,0.6,30.0,Y
996,671,876534,M,31,3.0,60,12.3,4.1,2.2,0.7,2.4,15.4,37.2,Y
997,669,87654,M,30,7.1,81,6.7,4.1,1.1,1.2,2.4,8.1,27.4,Y
998,99,24004,M,38,5.8,59,6.7,5.3,2.0,1.6,2.9,14.0,40.5,Y


In [11]:
diabetes_df = diabetes_df[(diabetes_df['AGE'] >= 25) & (diabetes_df['AGE'] <= 65) & (diabetes_df['BMI'] >= 18.5)]
diabetes_df.reset_index(drop = True, inplace = True)
diabetes_df

,ID,No_Pation,Gender,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI,CLASS
0,502,17975,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
1,735,34221,M,26,4.5,62,4.9,3.7,1.4,1.1,2.1,0.6,23.0,N
2,420,47975,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
3,680,87656,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
4,504,34223,M,33,7.1,46,4.9,4.9,1.0,0.8,2.0,0.4,21.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,198,454316,M,28,3.5,61,8.5,4.5,1.9,1.1,2.6,0.8,37.0,Y
942,671,876534,M,31,3.0,60,12.3,4.1,2.2,0.7,2.4,15.4,37.2,Y
943,669,87654,M,30,7.1,81,6.7,4.1,1.1,1.2,2.4,8.1,27.4,Y
944,99,24004,M,38,5.8,59,6.7,5.3,2.0,1.6,2.9,14.0,40.5,Y


In [14]:
diabetes_df = diabetes_df.copy()

# Remove empty spaces
diabetes_df.loc[:, 'CLASS'] = diabetes_df['CLASS'].str.strip()

# Replace "f" with "F"
diabetes_df.loc[:, 'Gender'] = diabetes_df['Gender'].str.replace('f', 'F')

In [15]:
# Get the frequency of each patient measure
repeated_measures = diabetes_df.groupby('No_Pation').size()

# Get the ids of the patients who have more than 1 measure
repeated_measures_ids = repeated_measures[repeated_measures > 1].index
repeated_measures_ids

Int64Index([   345,    856,   2345,  14389,  23972,  34325,  34514,  34515,
             34516,  34517,  34518,  34545,  44835,  45370,  45646,  71741,
             87654, 454316],
           dtype='int64', name='No_Pation')

In [17]:
# Keep just one observation by patient
diabetes_df = diabetes_df.drop_duplicates(subset = 'No_Pation', keep = 'first')

# Set the number of patient as the row index
diabetes_df.set_index('No_Pation', inplace = True, drop = True)
diabetes_df.head()

,ID,Gender,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI,CLASS
No_Pation,,,,,,,,,,,,,
17975,502,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
34221,735,M,26,4.5,62,4.9,3.7,1.4,1.1,2.1,0.6,23.0,N
47975,420,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
87656,680,F,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,N
34223,504,M,33,7.1,46,4.9,4.9,1.0,0.8,2.0,0.4,21.0,N


In [19]:
diabetes_df.duplicated().sum()

0

In [30]:
# Fit and transform the categorical columns
from sklearn.preprocessing import LabelEncoder
feature_column = 'Gender'
target_column = 'CLASS'

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

diabetes_df[feature_column] = label_encoder.fit_transform(diabetes_df[feature_column])

diabetes_df[target_column] = label_encoder.fit_transform(diabetes_df[target_column])
diabetes_df.head()

,ID,Gender,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI,CLASS
No_Pation,,,,,,,,,,,,,
17975,502,0,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,0
34221,735,1,26,4.5,62,4.9,3.7,1.4,1.1,2.1,0.6,23.0,0
47975,420,0,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,0
87656,680,0,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0,0
34223,504,1,33,7.1,46,4.9,4.9,1.0,0.8,2.0,0.4,21.0,0


In [49]:
diabetes_df.drop('ID', axis=1, inplace=True)


In [50]:
X = diabetes_df.iloc[:,:11]
y = diabetes_df.iloc[:,11]
X

,Gender,AGE,Urea,Cr,HbA1c,Chol,TG,HDL,LDL,VLDL,BMI
No_Pation,,,,,,,,,,,
17975,0,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0
34221,1,26,4.5,62,4.9,3.7,1.4,1.1,2.1,0.6,23.0
47975,0,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0
87656,0,50,4.7,46,4.9,4.2,0.9,2.4,1.4,0.5,24.0
34223,1,33,7.1,46,4.9,4.9,1.0,0.8,2.0,0.4,21.0
...,...,...,...,...,...,...,...,...,...,...,...
834578,1,51,5.5,83,9.4,6.4,2.3,2.3,3.2,1.0,33.0
4543,0,55,4.1,34,13.9,5.4,1.6,1.6,3.1,0.7,33.0
876534,1,31,3.0,60,12.3,4.1,2.2,0.7,2.4,15.4,37.2


In [51]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for PSO
def objective_function(features, X_train, X_test, y_train, y_test):
    features = np.round(features).astype(int)  # Ensure features are integers
    selected_features = X_train.iloc[:, features]
    
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(selected_features, y_train)
    
    # Evaluate on the test set
    selected_features_test = X_test.iloc[:, features]
    y_pred = clf.predict(selected_features_test)
    
    # Minimize negative accuracy (maximize accuracy)
    return -accuracy_score(y_test, y_pred)


# Define the bounds for the features
num_features = X.shape[1]
lb = [0] * num_features  # Lower bound for each feature
ub = [num_features - 1] * num_features  # Upper bound for each feature
bounds = (lb, ub)

# Use PSO to find the optimal set of features
best_features, _ = pso(objective_function, lb, ub, args=(X_train, X_test, y_train, y_test), swarmsize=10, maxiter=10)

# Print the best features selected by PSO
print("Best Features Selected:", best_features.astype(int))

Stopping search: maximum iterations reached --> 10
Best Features Selected: [ 5  4  0  0  5  4  6  4  0  4 10]


In [53]:

# Ensure unique features in the result
best_features = np.unique(best_features.astype(int))

# Print the best features selected by PSO
print("Best Features Selected:", best_features)


Best Features Selected: [ 0  4  5  6 10]


In [54]:
# Assuming X_train is a DataFrame
selected_column_names = X_train.columns[np.round(best_features).astype(int)].tolist()

# Print the best features selected by PSO
print("Best Features Selected:", best_features.astype(int))
print("Corresponding Column Names:", selected_column_names)

Best Features Selected: [ 0  4  5  6 10]
Corresponding Column Names: ['Gender', 'HbA1c', 'Chol', 'TG', 'BMI']
